In [54]:
from pykrx import stock
from pykrx import bond
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### pykrx의 종목 리스트 선정

일단 확인차 코스닥 상장 종목들의 ticker들을 불러와보자.

In [3]:
tickers = stock.get_market_ticker_list(market="KOSDAQ")

In [4]:
tickers

['060310',
 '054620',
 '265520',
 '211270',
 '126600',
 '013720',
 '035760',
 '311690',
 '051500',
 '058820',
 '023460',
 '056730',
 '065770',
 '083660',
 '404950',
 '367340',
 '367360',
 '060900',
 '025440',
 '068790',
 '241520',
 '065150',
 '180400',
 '245620',
 '037370',
 '050120',
 '214270',
 '130500',
 '900290',
 '083450',
 '297890',
 '078150',
 '195940',
 '028300',
 '278650',
 '067630',
 '024850',
 '047920',
 '115450',
 '403870',
 '036640',
 '405350',
 '426550',
 '439730',
 '442770',
 '448760',
 '095340',
 '099520',
 '950170',
 '067290',
 '035900',
 '318000',
 '024840',
 '024120',
 '021320',
 '036670',
 '044180',
 '151860',
 '046440',
 '035600',
 '226360',
 '111870',
 '060720',
 '058400',
 '114450',
 '060370',
 '122450',
 '052900',
 '376190',
 '061970',
 '309960',
 '086960',
 '038340',
 '160550',
 '053290',
 '060250',
 '104200',
 '030190',
 '024940',
 '218410',
 '327260',
 '091340',
 '419530',
 '019550',
 '950110',
 '046140',
 '151910',
 '036120',
 '099220',
 '036540',
 '255220',

 위에서 불러온 ticker 중에서 아무거나 샘플로 추출하여 종목명을 확인해보자.

In [5]:
sample = stock.get_market_ticker_name('065770')

In [6]:
sample

'CS'

확인을 해봤으니 이제 본격적으로 각 시장에서 ticker들과 종목명을 불러와 데이터 프레임으로 만들어보자.

In [55]:
market_list = ['KOSPI', 'KOSDAQ', 'KONEX']

In [56]:
ticker_df = pd.DataFrame()

In [57]:
for market_value in market_list : # 3개의 마켓 호출
    print(market_value)
    # 각 마켓에 대한 ticker_list 생성
    ticker_list = stock.get_market_ticker_list(market=market_value)
    for ticker_value in ticker_list : # 각 ticker_list 호출
        # 각 ticker에 대한 회사 이름 추출
        corp_name = stock.get_market_ticker_name(ticker_value)
        
        # 현재까지 얻은 정보
        # 마켓 이름, ticker, 회사 이름
        ticker_info_df = pd.DataFrame({
            'ticker':[ticker_value],
            'corp_name':[corp_name],
            'market':[market_value]
            
        })
        ticker_df = pd.concat([ticker_df,ticker_info_df])
        

KOSPI
KOSDAQ
KONEX


인덱스를 정리하여 최종 ticker_df으로 설정해주자

In [19]:
ticker_df = ticker_df.reset_index(drop=True)

만들어진 마스터 데이터 프레임을 저장하여 확인해보자

In [58]:
ticker_df.to_csv('master_frame.csv',index=False)

In [59]:
pd.read_csv('master_frame.csv')

,ticker,corp_name,market
0,095570,AJ네트웍스,KOSPI
1,006840,AK홀딩스,KOSPI
2,027410,BGF,KOSPI
3,282330,BGF리테일,KOSPI
4,138930,BNK금융지주,KOSPI
...,...,...,...
2713,150440,피노텍,KONEX
2714,242350,피엔아이컴퍼니,KONEX
2715,331660,한국미라클피플사,KONEX
2716,107640,한중엔시에스,KONEX


마스터 데이터 프레임에서 특정 종목(에코프로비엠) 하나를 선정하여 확인해보자. 여기서 추출방식은 ticker를 사용한다.

In [60]:
echo = ticker_df[ticker_df['ticker'] == '247540']

In [61]:
echo

,ticker,corp_name,market
0,247540,에코프로비엠,KOSDAQ


이제 원하는 종목들 6가지를 추출할 것이다.
여기서는 2차전지 관련 종목 3가지와 마이데이터 관련 종목 3가지를 정하여 각각을 추출할 것이다. 해당 종목의 ticker들은 증권홈페이지에서 쉽게 찾을 수 있다.

- 삼성SDI : 006400 
- LG에너지솔루션 : 373220 
- 에코프로비엠 : 247540
- NICE평가정보: 030190 
- 파이오링크: 170790 
- 헥토이노베이션: 214180

ticker로 해당 종목을 추출하는 방식은 여러가지가 있는데, 그 중 두가지 방법으로 추출을 해보려고 한다.
하나는 반복문을 통한 추출이다.

In [67]:
battery_ticker = pd.DataFrame()  #2차전지 관련주 추출

In [68]:
sample_ticker_list = ['006400','373220','247540']

In [69]:
for sample_list in sample_ticker_list:
    print(sample)
    df_battery =  ticker_df[ticker_df['ticker'] == sample_list]
    battery_ticker = pd.concat([battery_ticker,df_battery])

CS
CS
CS


In [70]:
battery_ticker

,ticker,corp_name,market
0,006400,삼성SDI,KOSPI
0,373220,LG에너지솔루션,KOSPI
0,247540,에코프로비엠,KOSDAQ


또 다른 하나는 contains 메소드를 활용한 방법이다.

In [73]:
battery_ticker = ticker_df[ticker_df['ticker'].str.contains('006400|373220|247540')] # '|'는 or을 뜻한다. 
battery_ticker

,ticker,corp_name,market
0,373220,LG에너지솔루션,KOSPI
0,006400,삼성SDI,KOSPI
0,247540,에코프로비엠,KOSDAQ


인덱스를 정리해주자

In [78]:
battery_ticker = battery_ticker.reset_index(drop=True)
battery_ticker

,ticker,corp_name,market
0,373220,LG에너지솔루션,KOSPI
1,006400,삼성SDI,KOSPI
2,247540,에코프로비엠,KOSDAQ


위 2가지 방법 중 개수가 많지 않을 경우에는 후자를 더 추천한다.

이번에는 같은 방식으로 마이데이터 관련주를 추출해보자.

In [75]:
mydata_ticker = ticker_df[ticker_df['ticker'].str.contains('030190|170790|214180')]
mydata_ticker

,ticker,corp_name,market
0,030190,NICE평가정보,KOSDAQ
0,170790,파이오링크,KOSDAQ
0,214180,헥토이노베이션,KOSDAQ


In [80]:
mydata_ticker = mydata_ticker.reset_index(drop=True)
mydata_ticker

,ticker,corp_name,market
0,030190,NICE평가정보,KOSDAQ
1,170790,파이오링크,KOSDAQ
2,214180,헥토이노베이션,KOSDAQ
